In [1]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback

In [2]:
def get_raw_data_go(filename):
    ''' Returns the set of samples from the local file or download it if it does not exists'''
    import gzip, os.path
    import json

    raw_samples_file = filename

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        import urllib.request 
        urllib.request.urlretrieve("https://www.labri.fr/perso/lsimon/ia-inge2/"+filename, filename)
        print(" Done")

    with gzip.open(filename) as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data

data = get_raw_data_go("samples-9x9.json.gz")
print("We have", len(data),"examples")

We have 41563 examples


In [3]:
def name_to_coord(s):
    assert s != "PASS"
    indexLetters = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'J':8}

    col = indexLetters[s[0]]
    lin = int(s[1:]) - 1
    return col, lin


def stones_to_tensor(black_stones, white_stones) :
    # insérer les pions noirs
    B = np.zeros((9, 9))
    for p in black_stones :
        i, j = name_to_coord(p)
        B[i, j] = 1
    # insérer les pions blancs
    W = np.zeros((9, 9))
    for p in white_stones :
        i, j = name_to_coord(p)
        W[i, j] = 1
        
    return np.array([B,W]).reshape(9,9,2)


#Vous prendrez en entrée data[i]["black_stones"] et data[i]["white_stones"]
X_tmp = np.array([stones_to_tensor(stones["black_stones"], stones["white_stones"]) for stones in data])
#Vous devrez prédire simplement data[i]["black_wins"]/data[i]["rollouts"]
y_tmp = np.array([d["black_wins"] / d["rollouts"] for d in data])

print(X_tmp.shape)
print(y_tmp.shape)

# ajouter les symétries et les rotations
# 12  24  43  31
# 34  13  21  42
# 
# 34  13  21  42
# 12  24  43  31

X = list()
y = list()

for i in range(len(X_tmp)) :
    new = list()
    new.append(X_tmp[i].reshape(2,9,9))
    new.append(np.array([np.flipud(new[-1][0]), np.flipud(new[-1][1])]))
    new.append(np.array([np.rot90(new[-2][0]), np.rot90(new[-2][1])]))
    new.append(np.array([np.flipud(new[-1][0]), np.flipud(new[-1][1])]))
    new.append(np.array([np.rot90(new[-2][0]), np.rot90(new[-2][1])]))
    new.append(np.array([np.flipud(new[-1][0]), np.flipud(new[-1][1])]))
    new.append(np.array([np.rot90(new[-2][0]), np.rot90(new[-2][1])]))
    new.append(np.array([np.flipud(new[-1][0]), np.flipud(new[-1][1])]))   
    X += [r.reshape(9,9,2) for r in new]
    y += [y_tmp[i]] * 8
    # ajouter en inversant noirs/blancs 
    X += [np.array([r[1], r[0]]).reshape(9,9,2) for r in new]
    y += [1-y_tmp[i]] * 8

       
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)   

(41563, 9, 9, 2)
(41563,)
(665008, 9, 9, 2)
(665008,)


In [4]:
def model_julien():
    λ = optimizers.schedules.InverseTimeDecay( # learning rate
        initial_learning_rate=1e-3,
        decay_steps=15000,
        decay_rate=0.5)

    α = 0.4 # dropout rate

    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(9,9,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(rate=α))
    model.add(Dense(648, activation='relu'))
    model.add(Dropout(rate=α))
    model.add(Dense(324, activation='relu'))
    model.add(Dense(162, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(42, activation='relu'))
    model.add(Dense(21, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizers.RMSprop(λ), loss='mean_absolute_error')

    # optimizer : optimizers.RMSprop(λ) ; optimizers.Adam(learning_rate=λ) ; optimizers.SGD(learning_rate=λ)
    # loss : binary_crossentropy ; mean_squared_error ; mean_absolute_error

    model.summary()
    return model


def tuned_model(): #Tuned with Keras Tuner
    model = Sequential()
    model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=(9,9,2), padding ="same"))
    model.add(Conv2D(256, kernel_size=5, activation='relu', padding ="same"))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(2, activation='softmax')) 
    opt= optimizers.Adam()
    model.compile(loss='mean_absolute_error',
                optimizer = opt,
                metrics=['accuracy'])
    model.summary()
    return model

model = model_julien()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 32)          608       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 32)          128       
_________________________________________________________________
flatten (Flatten)            (None, 2592)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2592)              0         
_________________________________________________________________
dense (Dense)                (None, 648)               1680264   
_________________________________________________________________
dropout_1 (Dropout)          (None, 648)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 324)               2

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# ==================================================================================
class History(Callback):
    
    def __init__(self):
        self.history = {}
        
    def on_epoch_end(self, epoch, logs={}):       
        for k,v in logs.items():
            if not k in self.history: self.history[k]=[]
            self.history[k].append(v)
        print(".",end="")
history=[History()]

In [6]:
#es_callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# hyperparamètres
epochs = 50
batch_size = 200

# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          callbacks=history)

Training ...
Epoch 1/50
2228/2228 [==============================] - 58s 26ms/step - loss: 0.1683 - val_loss: 0.1356
Epoch 2/50
2228/2228 [==============================] - 53s 24ms/step - loss: 0.1446 - val_loss: 0.1272
Epoch 3/50
2228/2228 [==============================] - 53s 24ms/step - loss: 0.1387 - val_loss: 0.1256
Epoch 4/50
2228/2228 [==============================] - 55s 25ms/step - loss: 0.1344 - val_loss: 0.1254
Epoch 5/50
2228/2228 [==============================] - 58s 26ms/step - loss: 0.1317 - val_loss: 0.1206
Epoch 6/50
2228/2228 [==============================] - 54s 24ms/step - loss: 0.1298 - val_loss: 0.1171
Epoch 7/50
2228/2228 [==============================] - 60s 27ms/step - loss: 0.1281 - val_loss: 0.1159
Epoch 8/50
2228/2228 [==============================] - 60s 27ms/step - loss: 0.1262 - val_loss: 0.1173
Epoch 9/50
2228/2228 [==============================] - 57s 26ms/step - loss: 0.1251 - val_loss: 0.1162
Epoch 10/50
2228/2228 [============================

In [7]:
# testing
print("After training :")
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)

nb_to_test = 14

print("Verification on train : ")
print("expected : ", y_train[0:nb_to_test])
print("got      : ", model.predict(X_train[0:nb_to_test]).reshape(1,nb_to_test)[0].round(2))

print("Verification on test : ")
print("expected : ", y_test[0:nb_to_test])
print("got      : ", model.predict(X_test[0:nb_to_test]).reshape(1,nb_to_test)[0].round(2))

# Model accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss (on training data)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Loss', 'Loss (sur validation)'])
plt.show()

After training :
Test loss: 0.10642054677009583
Verification on train : 
expected :  [0.51 0.89 0.52 0.83 0.42 0.37 1.   0.   0.09 0.82 0.63 0.   0.21 0.41]
got      :  [0.46 0.72 0.47 0.76 0.35 0.37 0.87 0.09 0.21 0.87 0.48 0.   0.22 0.48]
Verification on test : 
expected :  [0.68 1.   0.23 0.   0.01 0.   0.   1.   1.   0.16 0.08 0.02 0.4  0.21]
got      :  [0.4  1.   0.1  0.   0.   0.   0.11 1.   1.   0.02 0.1  0.05 0.5  0.46]


NameError: name 'plt' is not defined

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(1, y_test.shape[0])[0]
    
def print_repartition(y_pred, y_test) :
    delta = abs(y_pred - y_test)
    stat = [0] * 20
    for val in delta :
        for i in range(20) :
            if val <= 0.05*(i+1) :
                stat[i] += 1
                break
    for i in range(20) :
        print("val <= " + str(np.round(0.05*(i+1),3)) + " : ", 100*stat[i]/len(y_test), "%")
        
print_repartition(y_pred, y_test)